In [5]:
import requests

# 中央氣象署 API 授權碼
cwa_api_key = "CWA-13E0218C-0EE7-4EC8-BBA4-6A8E26DEB6DF"
# API 代碼 (F-C0032-001 = 縣市天氣預報資料)
data_id = "F-C0032-001"

url = f"https://opendata.cwa.gov.tw/api/v1/rest/datastore/{data_id}"
params = {"Authorization": cwa_api_key, "locationName": '臺北市', "format": "JSON"}
response = requests.get(url, params=params)
data = response.json()
print(data)

{'success': 'true', 'result': {'resource_id': 'F-C0032-001', 'fields': [{'id': 'datasetDescription', 'type': 'String'}, {'id': 'locationName', 'type': 'String'}, {'id': 'parameterName', 'type': 'String'}, {'id': 'parameterValue', 'type': 'String'}, {'id': 'parameterUnit', 'type': 'String'}, {'id': 'startTime', 'type': 'Timestamp'}, {'id': 'endTime', 'type': 'Timestamp'}]}, 'records': {'datasetDescription': '三十六小時天氣預報', 'location': [{'locationName': '臺北市', 'weatherElement': [{'elementName': 'Wx', 'time': [{'startTime': '2025-10-05 18:00:00', 'endTime': '2025-10-06 06:00:00', 'parameter': {'parameterName': '晴時多雲', 'parameterValue': '2'}}, {'startTime': '2025-10-06 06:00:00', 'endTime': '2025-10-06 18:00:00', 'parameter': {'parameterName': '晴時多雲', 'parameterValue': '2'}}, {'startTime': '2025-10-06 18:00:00', 'endTime': '2025-10-07 06:00:00', 'parameter': {'parameterName': '晴時多雲', 'parameterValue': '2'}}]}, {'elementName': 'PoP', 'time': [{'startTime': '2025-10-05 18:00:00', 'endTime': '20

In [1]:
import requests

# 中央氣象署 API 授權碼
cwa_api_key = "CWA-13E0218C-0EE7-4EC8-BBA4-6A8E26DEB6DF"
# API 代碼 (F-C0032-001 = 縣市天氣預報資料)
data_id = "F-C0032-001"

def get_cwa_weather(location_name):
    """
    從中央氣象署 API 取得指定縣市的天氣資料
    """
    url = f"https://opendata.cwa.gov.tw/api/v1/rest/datastore/{data_id}"
    params = {"Authorization": cwa_api_key, "locationName": location_name, "format": "JSON"}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # 如果 HTTP 狀態碼錯誤，會丟出例外
        data = response.json()

        # 從 JSON 中找到 location 資料
        location_data = data.get("records", {}).get("location", [])
        if not location_data:
            print(f"找不到「{location_name}」的天氣資料")
            return None
        return location_data[0]

    except Exception as e:
        print(f"API 請求失敗：{e}")
        return None


def display_cwa_weather(location_data):
    """
    顯示縣市天氣資料
    """
    if not location_data:
        print("⚠ 沒有天氣資料")
        return

    # 取得地區名稱
    location_name = location_data.get("locationName", "未知地區")

    # 將各項天氣資料整理成字典
    forecast = {}
    for element in location_data.get("weatherElement", []):
        element_name = element.get("elementName")
        first_period = element.get("time", [{}])[0]  # 只取第一個時段
        parameter = first_period.get("parameter", {})
        forecast[element_name] = {
            "start_time": first_period.get("startTime", "N/A"),
            "end_time": first_period.get("endTime", "N/A"),
            "value": parameter.get("parameterName", "N/A"),
            "unit": parameter.get("parameterUnit", "")
        }

    # 輸出結果
    print("\n--- 中央氣象署天氣預報 ---")
    print(f" 地點：{location_name}")
    print(f" 預報時段：{forecast.get('Wx', {}).get('start_time')} 至 {forecast.get('Wx', {}).get('end_time')}")
    print("--------------------------")
    print(f" 天氣狀況：{forecast.get('Wx', {}).get('value')}")
    print(f" 溫度範圍：{forecast.get('MinT', {}).get('value')}°{forecast.get('MinT', {}).get('unit')} - "
          f"{forecast.get('MaxT', {}).get('value')}°{forecast.get('MaxT', {}).get('unit')}")
    print(f" 降雨機率：{forecast.get('PoP', {}).get('value')}{forecast.get('PoP', {}).get('unit')}")
    print(f" 舒適度：{forecast.get('CI', {}).get('value')}")
    print("--------------------------")


# --- 主程式 ---
if __name__ == "__main__":
    user_input = input("\n請輸入想查詢的縣市名稱 (例如: 臺北市, 桃園市, 高雄市): ")
    if user_input:
        data = get_cwa_weather(user_input)
        display_cwa_weather(data)



請輸入想查詢的縣市名稱 (例如: 臺北市, 桃園市, 高雄市):  桃園市



--- 中央氣象署天氣預報 ---
 地點：桃園市
 預報時段：2025-10-05 18:00:00 至 2025-10-06 06:00:00
--------------------------
 天氣狀況：晴時多雲
 溫度範圍：26°C - 30°C
 降雨機率：10百分比
 舒適度：舒適至悶熱
--------------------------
